In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
data_source = r'c:\users\sivin\appdata\local\programs\python\python37-32\Scripts\Stock\trainset.csv'
data = pd.read_csv(data_source)
data_source2 = r'c:\users\sivin\appdata\local\programs\python\python37-32\Scripts\Stock\testset.csv'
data2 = pd.read_csv(data_source2 )
print(data.head())
print(data2.head())
trainer = data
tester = data2
#print(trainer.head())
#print(tester.head())
trainer = trainer.drop(['Date', 'Adj Close'], axis = 1)
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(trainer)

X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)
dataset_test = tester
real_stock_price = dataset_test.iloc[:, 1:2].values
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 76):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
plt.plot(real_stock_price, color = 'black', label = 'TATA Stock Price')
plt.plot(predicted_stock_price, color = 'green', label = 'Predicted TATA Stock Price')
plt.title('TATA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TATA Stock Price')
plt.legend()
plt.show()


Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\users\sivin\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow_internal', [dirname(__file__)])
  File "c:\users\sivin\appdata\local\programs\python\python37-32\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow_internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\sivin\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\sivin\appdata\local\programs\python\python37-32\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
   

ValueError: cannot reshape array of size 300 into shape (1,60,1)

In [20]:
#import the dataset

#train_set = df.iloc[:,1:2].values

#scale to fit b/w 0 & 1

#from sklearn.preprocessing import MinMaxScaler
#sc = MinMaxScaler(feature_range = (0,1))
#scaled_train_set = sc.fit_transform(train_set)

#start = dt.datetime(1985,1,29)
#end = dt.datetime.today()
#stock ='^DJI'
#df = web.DataReader(stock,'yahoo',start,end)
#df = df.rename(columns={'Adj Close':'close'})
#df = df.loc[:,['Open','High','Low','close','Volume']]